---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize, scale

ModuleNotFoundError: No module named 'sklearn'

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [30]:
def graph_identification():
    
    # Your Code Here
    graph_identification_list =[]
    
    for G in range(len(P1_Graphs)):
        Graph = P1_Graphs[G]
        
        avg_clt = nx.average_clustering(Graph)
        avg_path = nx.average_shortest_path_length(Graph)
        #print(avg_clt, avg_spath)
        
        degrees = P1_Graphs[G].degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(j) / nx.number_of_nodes(P1_Graphs[G]) for j in degree_values]
        #print(histogram)
        
        if len(histogram)>10:
            graph_identification_list.append('PA')
            
        elif avg_clt < 0.1:
            graph_identification_list.append('SW_H')
            
        else:
            graph_identification_list.append('SW_L')
    
    return graph_identification_list # Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [35]:
df = pd.DataFrame(index=G.nodes())
df

""
0
1
2
3
4
...
1000
1001
1002
1003


In [49]:
df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
df['y'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))

In [50]:
df

,Department,y
0,1,0.0
1,1,NaN
2,21,NaN
3,21,1.0
4,21,1.0
...,...,...
1000,4,NaN
1001,21,NaN
1002,1,0.0
1003,6,0.0


In [51]:
df2_y = df.loc[~(df.y.isnull()), ['y']]

In [52]:
df2_y

,y
0,0.0
3,1.0
4,1.0
6,1.0
7,0.0
...,...
998,0.0
999,0.0
1002,0.0
1003,0.0


In [41]:
mask = df['y'].isnull()

In [42]:
mask

0       False
1        True
2        True
3       False
4       False
        ...  
1000     True
1001     True
1002    False
1003    False
1004    False
Name: y, Length: 1005, dtype: bool

In [43]:
df_pred = df[mask]

In [44]:
df_pred

,Department,y
1,1,NaN
2,21,NaN
5,25,NaN
8,14,NaN
14,4,NaN
...,...,...
992,4,NaN
994,21,NaN
996,14,NaN
1000,4,NaN


In [46]:
df_main = df[~mask]

In [47]:
df_main

,Department,y
0,1,0.0
3,21,1.0
4,21,1.0
6,25,1.0
7,14,0.0
...,...,...
998,14,0.0
999,15,0.0
1002,1,0.0
1003,6,0.0


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [34]:
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    
    df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
    df['y'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    
    df['ClustCoef'] = pd.Series(nx.clustering(G))
    df['DegreeCent'] = pd.Series(nx.degree_centrality(G))
    df['Degree'] = pd.Series(nx.degree(G))
    df['BetwCent'] = pd.Series(nx.betweenness_centrality(G))
    df['page_rank'] = pd.Series(nx.pagerank(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    
    columns = ['clustering', 'degree', 'closeness', 'betweenness', 'page_rank']
    
    missing = df['ManagementSalary'].isnull()]
    df_train = df[~missing]
    df_test = df[missing]
    
    X_train = df_train[columns]
    Y_train = df_train['ManagementSalary']
    X_test = df_test[columns]

    clf = DecisionTreeClassifier(max_depth=5)
    clf.fit(X_train, y_train)
    
    prob = clf.predict_proba(X_test)[:, 1]
    
    
    return pd.Series(prob, index=X_test.index)  # Your Answer Here
salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [55]:
df = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
G = nx.read_gpickle('email_prediction.txt')

In [81]:
 df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]

In [82]:
df['preferential attachment']

(6, 840)      2070
(4, 197)      3552
(620, 979)      28
(519, 872)     299
(382, 423)     205
              ... 
(165, 923)     714
(673, 755)       3
(939, 940)       6
(555, 905)     160
(75, 101)      638
Name: preferential attachment, Length: 488446, dtype: int64

In [72]:
df['Common Neighbors'] = df.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))

In [73]:
df['Common Neighbors']

(6, 840)      9
(4, 197)      2
(620, 979)    0
(519, 872)    2
(382, 423)    0
             ..
(165, 923)    0
(673, 755)    0
(939, 940)    0
(555, 905)    0
(75, 101)     1
Name: Common Neighbors, Length: 488446, dtype: int64

In [75]:
df['common_neigh'] = [len(list(nx.common_neighbors(G, e[0], e[1]))) for e in df.index.values]

In [76]:
df['common_neigh']

(6, 840)      9
(4, 197)      2
(620, 979)    0
(519, 872)    2
(382, 423)    0
             ..
(165, 923)    0
(673, 755)    0
(939, 940)    0
(555, 905)    0
(75, 101)     1
Name: common_neigh, Length: 488446, dtype: int64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    df = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    
    G = nx.read_gpickle('email_prediction.txt')
    
    df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    
    df['Common Neighbors'] = [len(list(nx.common_neighbors(G, e[0], e[1]))) for e in df.index.values]
    
    df['jacard'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    
    df['community common neighbors'] = (
        [i[2] for i in nx.cn_soundarajan_hopcroft(G, community='Department', ebunch=df.index)])
    
    df['community resource allocation'] = (
        [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, community='Department', ebunch=df.index)])
    
    columns = [
        'preferential attachment', 
        'Common Neighbors',
        'jacard', 
        'community common neighbors',
        'community resource allocation'
    ]
    
    missing = df['Future Connection'].isnull()
    df_train = df[~missing]
    df_test = df[missing]
    
    X_train = df_train[columns]
    Y_train = df_train['Future Connection']
    X_test = df_test[columns]
    
    clf = DecisionTreeClassifier(max_depth=5)
    clf.fit(X_train, y_train)
    
    prob = clf.predict_proba(X_test)[:, 1]   
    
    return pd.Series(prob, index=X_test.index) # Your Answer Here
new_connections_predictions()